# Upload files to OpenAI

In [ ]:
import requests
import os

# Load OPENAI_API_KEY from environment variables
API_KEY = "OpenAI key"

# Define the API endpoint
url = "https://api.openai.com/v1/files"

# Specify the file paths
file_paths = [
    "fortune_train.jsonl",
    "fortune_val.jsonl"
]

# Set headers
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

In [10]:
# Upload each file
for file_path in file_paths:
    with open(file_path, "rb") as file:
        files = {
            "file": file
        }
        data = {
            "purpose": "fine-tune"
        }

        response = requests.post(url, headers=headers, files=files, data=data)

        # Print response for each file
        print(f"Uploaded {file_path}:")
        print(response.json())

Uploaded fortune_train.jsonl:
{'object': 'file', 'id': 'file-EnyTp8JPFitbTCKiPTD3Fs', 'purpose': 'fine-tune', 'filename': 'fortune_train.jsonl', 'bytes': 286683, 'created_at': 1747690796, 'expires_at': None, 'status': 'processed', 'status_details': None}
Uploaded fortune_val.jsonl:
{'object': 'file', 'id': 'file-8C5JwpG71m7hLB6XV8h5bE', 'purpose': 'fine-tune', 'filename': 'fortune_val.jsonl', 'bytes': 67347, 'created_at': 1747690797, 'expires_at': None, 'status': 'processed', 'status_details': None}



# Fine tune the model

In [4]:
MODEL = "gpt-4o-mini-2024-07-18"

import openai

# load OPENAI_API_KEY from environment variables
client = openai.Client(api_key=API_KEY)

response = client.fine_tuning.jobs.create(
    training_file='file-EnyTp8JPFitbTCKiPTD3Fs',
    validation_file='file-8C5JwpG71m7hLB6XV8h5bE',
    model=MODEL,
    suffix="design-voice",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-zs8qr2yk3YkVbnxcBTDv0Ggi
Status: validating_files


# Evaluation Metrics

In [12]:
result = client.fine_tuning.jobs.retrieve("ftjob-zs8qr2yk3YkVbnxcBTDv0Ggi")
print(result)

FineTuningJob(id='ftjob-zs8qr2yk3YkVbnxcBTDv0Ggi', created_at=1747691547, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:design-voice:BZ2wbUFw', finished_at=1747692495, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-HZTMyl5DL9ug1km6SpZTENRJ', result_files=['file-CSp6sDN8sGgqnvf588RuN9'], seed=923395405, status='succeeded', trained_tokens=241293, training_file='file-EnyTp8JPFitbTCKiPTD3Fs', validation_file='file-8C5JwpG71m7hLB6XV8h5bE', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3))), user_provided_suffix='design-voice', usage_metrics=None, shared_with_openai=False, eval_id=None)


In [13]:
# Define the API endpoint
job_id = "ftjob-zs8qr2yk3YkVbnxcBTDv0Ggi"
url = f"https://api.openai.com/v1/fine_tuning/jobs/{job_id}/checkpoints"

# Set headers
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

# Send GET request
response = requests.get(url, headers=headers)

# Print response
print(response.json())

{'object': 'list', 'data': [{'object': 'fine_tuning.job.checkpoint', 'id': 'ftckpt_xdSwmM2YT9ypHP8DEe0OTTaQ', 'created_at': 1747692445, 'fine_tuned_model_checkpoint': 'ft:gpt-4o-mini-2024-07-18:personal:design-voice:BZ2wbUFw', 'fine_tuning_job_id': 'ftjob-zs8qr2yk3YkVbnxcBTDv0Ggi', 'metrics': {'step': 495, 'type': 'base'}, 'step_number': 495}, {'object': 'fine_tuning.job.checkpoint', 'id': 'ftckpt_8CliWithej402Wc9ChdYC3xz', 'created_at': 1747692226, 'fine_tuned_model_checkpoint': 'ft:gpt-4o-mini-2024-07-18:personal:design-voice:BZ2wbFgN:ckpt-step-330', 'fine_tuning_job_id': 'ftjob-zs8qr2yk3YkVbnxcBTDv0Ggi', 'metrics': {'step': 330, 'type': 'base'}, 'step_number': 330}, {'object': 'fine_tuning.job.checkpoint', 'id': 'ftckpt_riHLhT82xw4FEXkZgMKfkqjg', 'created_at': 1747692000, 'fine_tuned_model_checkpoint': 'ft:gpt-4o-mini-2024-07-18:personal:design-voice:BZ2waIXk:ckpt-step-165', 'fine_tuning_job_id': 'ftjob-zs8qr2yk3YkVbnxcBTDv0Ggi', 'metrics': {'step': 165, 'type': 'base'}, 'step_numbe